In [1]:
# 导包
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

import os


In [2]:
# path
df_row_data_dir_path = os.path.abspath(os.path.dirname(os.path.dirname(os.getcwd())))

df_row_data_path1 = r'\new_data\kmer\train_1_meter.csv'
df_row_data_path2 = r'\new_data\kmer\train_2_meter.csv'
df_row_data_path3 = r'\new_data\kmer\train_3_meter.csv'
df_row_data_path4 = r'\new_data\kmer_gap\train3_gap_1_1_meter.csv'


In [3]:
# 数据载入
df_data1 = pd.read_csv(df_row_data_dir_path+df_row_data_path1)
df_data2 = pd.read_csv(df_row_data_dir_path+df_row_data_path2).drop('label',axis = 1)
df_data3 = pd.read_csv(df_row_data_dir_path+df_row_data_path3).drop('label',axis = 1)
df_data4 = pd.read_csv(df_row_data_dir_path+df_row_data_path4).drop('label',axis = 1)

In [4]:
# 将数据合并，前conmcat_num个数据合并
concat_num = 4
#concat_list = [df_data1,df_data2,df_data3,df_data4,df_data5,df_data6,df_data7]
concat_list = [df_data1,df_data2,df_data3,df_data4]

df_row_train = pd.concat(concat_list[:concat_num], axis = 1)

In [5]:
df_row_train.head()

,label,A,C,D,E,F,G,H,I,K,...,T*D,L*L,W*Y,D*C,L*P,M*D,K*D,V*H,C*V,E*Q
0,1,6,1,12,15,9,9,2,8,22,...,2,2,0,0,1,0,0,1,0,0
1,1,63,2,46,80,27,58,13,62,76,...,4,5,0,0,3,1,1,1,0,1
2,1,14,7,5,11,3,5,0,6,13,...,0,1,0,1,1,0,1,0,1,1
3,1,14,1,3,9,3,12,3,9,6,...,0,1,0,0,0,0,0,0,0,0
4,1,2,0,16,15,5,3,0,9,13,...,2,0,0,0,0,0,2,0,0,2


In [6]:
df_data_label = df_row_train.label
df_data_features = df_row_train.drop('label',axis = 1)

In [7]:
df_data_features.head()

,A,C,D,E,F,G,H,I,K,L,...,T*D,L*L,W*Y,D*C,L*P,M*D,K*D,V*H,C*V,E*Q
0,6,1,12,15,9,9,2,8,22,18,...,2,2,0,0,1,0,0,1,0,0
1,63,2,46,80,27,58,13,62,76,59,...,4,5,0,0,3,1,1,1,0,1
2,14,7,5,11,3,5,0,6,13,5,...,0,1,0,1,1,0,1,0,1,1
3,14,1,3,9,3,12,3,9,6,11,...,0,1,0,0,0,0,0,0,0,0
4,2,0,16,15,5,3,0,9,13,8,...,2,0,0,0,0,0,2,0,0,2


In [8]:
from sklearn.feature_selection import VarianceThreshold, SelectPercentile
from sklearn.feature_selection import chi2

# 方差选择法，返回值为特征选择后的数据，参数threshold为方差的阈值

#selector = VarianceThreshold(threshold =5)
#new_data = selector.fit_transform(df_data_features)

selector = SelectPercentile(chi2,percentile=10)
new_data = selector.fit_transform(df_data_features, df_data_label)

new_data.shape

(1708, 1164)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [10]:
train_X,test_X,train_Y,test_Y=train_test_split(new_data,df_data_label, test_size=0.1, random_state=0)

## Model

In [11]:
tuned_parameters = [{'kernel':['rbf'],'gamma':[1e-3, 1e-4],
                    'C':[0.1,1,10,100,1000]},
                  { 'kernel':['linear'],'C':[1,10,100,1000]}]

In [12]:
# scores = ['precision','recall']
scores = ['precision']

In [ ]:
for score in scores:
    print('# Tuning hyper-parameters for %s' % score)
    print()
    
    # 调用GridSearchCV,将SVC()，tuned_parameters,cv = 10, 还有scoring传递进去
    clf = GridSearchCV(SVC(), tuned_parameters, cv = 10,
                      scoring = '%s_macro' % score)
    # 用训练集训练这个学习器 clf
    clf.fit(train_X,train_Y)
    
    print('Best paramters set found on development set:')
    print()
    
    # 再调用 clf.best_params_ 就能直接得到最好的参数搭配结果
    print(clf.best_params_)
    
    print()
    print('Grid scores on development set:')
    print()
    means = clf.cv_results_['mean_test_score']
    std = clf.cv_results_['std_test_score']
    
    # 看一下具体的参数间不同数值的组合后得到的分数是多少
    for mean, std, params in zip(means, std, clf.cv_results_['params']):
        print('%0.3f (+/-%0.03f) for %r'
             % (mean, std * 2, params))
    
    print()
    
    print('Detailed classification report:')
    print()
    print('The model is trained on the full development set.')
    print('The scores are computed on the full evaluation set.')
    print()
    y_ture, y_pred = test_Y, clf.predict(test_X)
    
    # 打印在测试集上的预测结果与真实值的分数
    print(classification_report(y_ture, y_pred))
    
    print()

# Tuning hyper-parameters for precision

